In [2]:
# IMPORTS
import pandas as pd
import math
import os.path
import time
from binance.client import Client
from datetime import timedelta, datetime
from dateutil import parser
from tqdm import tqdm_notebook #(Optional, used for progress-bars)

In [3]:
### API
binance_api_key = '0YoXpNjk2J0rwPmIDFlgRuu2fFHKHSADD6qUNoRVTr3N9Rddjdbg3AFP7jzyxvly'    #Enter your own API-key here
binance_api_secret = 'rKJZ8XQZGI4D9lTmYztoHmTDPw4fWcNWbT4vEs6edw8qEmBP4QqAYfBmxH6KGWiW' #Enter your own API-secret here

In [4]:
### CONSTANTS
binsizes = {"1m": 1, "5m": 5, "1h": 60, "1d": 1440}
batch_size = 750
client = Client("0YoXpNjk2J0rwPmIDFlgRuu2fFHKHSADD6qUNoRVTr3N9Rddjdbg3AFP7jzyxvly")

In [5]:
### FUNCTIONS
def minutes_of_new_data(symbol, kline_size, data, source):
    if len(data) > 0:  
        old = parser.parse(data["timestamp"].iloc[-1])
    elif source == "binance": 
        old = datetime.strptime('1 Jan 2017', '%d %b %Y')
    if source == "binance": 
        new = pd.to_datetime(
            client.get_klines(symbol=symbol, 
                                      interval=kline_size)[-1][0], 
            unit='ms'
        )
    return old, new

In [6]:
def get_all_binance(symbol, kline_size, save = False):
    filename = '%s-%s-data.csv' % (symbol, kline_size)
    if os.path.isfile(filename): 
        data_df = pd.read_csv(filename)
    else: 
        data_df = pd.DataFrame()
    oldest_point, newest_point = minutes_of_new_data(symbol, 
                                                     kline_size, 
                                                     data_df, 
                                                     source = "binance")
    delta_min = (newest_point - oldest_point).total_seconds()/60
    available_data = math.ceil(delta_min/binsizes[kline_size])
    if oldest_point == datetime.strptime('1 Jan 2017', '%d %b %Y'): 
        print('Downloading all available %s data for %s. \
        Be patient..!' % (kline_size, symbol))
    else: 
        print('Downloading %d minutes of new data available \
        for %s, i.e. %d instances of %s data.' \
              % (delta_min, symbol, available_data, kline_size))
    klines = client.get_historical_klines(symbol, 
                                          kline_size, 
                                          oldest_point.strftime("%d %b %Y %H:%M:%S"), 
                                          newest_point.strftime("%d %b %Y %H:%M:%S"))
    data = pd.DataFrame(klines, 
                        columns = ['timestamp', 'open', 'high', 'low', 'close', 'volume', 
                                   'close_time', 'quote_av', 'trades', 'tb_base_av', 
                                   'tb_quote_av', 'ignore' ])
    data['timestamp'] = pd.to_datetime(data['timestamp'], unit='ms')
    if len(data_df) > 0:
        temp_df = pd.DataFrame(data)
        data_df = data_df.append(temp_df)
    else: 
        data_df = data
    data_df.set_index('timestamp', inplace=True)
    if save: 
        data_df.to_csv(filename)
    print('All caught up..!')
    return data_df

In [7]:
def get_delta_binance(symbol, kline_size, save = False, delta=timedelta(days=7)):
    filename = '%s-%s-data.csv' % (symbol, kline_size)
    data_df = pd.DataFrame()
    oldest_point, newest_point = minutes_of_new_data(symbol, 
                                                     kline_size, 
                                                     data_df, 
                                                     source = "binance")
    oldest_point = newest_point - delta
    delta_min = (newest_point - oldest_point).total_seconds()/60
    available_data = math.ceil(delta_min/binsizes[kline_size])
    print('Downloading %d minutes of new data available \
    for %s, i.e. %d instances of %s data.' \
          % (delta_min, symbol, available_data, kline_size))
    klines = client.get_historical_klines(symbol, 
                                          kline_size, 
                                          oldest_point.strftime("%d %b %Y %H:%M:%S"), 
                                          newest_point.strftime("%d %b %Y %H:%M:%S"))
    data = pd.DataFrame(klines, 
                        columns = ['timestamp', 'open', 'high', 'low', 'close', 'volume', 
                                   'close_time', 'quote_av', 'trades', 'tb_base_av', 
                                   'tb_quote_av', 'ignore' ])
    data['timestamp'] = pd.to_datetime(data['timestamp'], unit='ms')
    if len(data_df) > 0:
        temp_df = pd.DataFrame(data)
        data_df = data_df.append(temp_df)
    else: 
        data_df = data
    data_df.set_index('timestamp', inplace=True)
    if save: 
        data_df.to_csv(filename)
    print('All caught up..!')
    return data_df

In [11]:
# For Binance
binance_symbols = ["BTCUSDT"]
for symbol in binance_symbols:
    get_delta_binance(symbol, '1h', save = True, delta=timedelta(days=60))

All caught up..!
